In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName("NYSE").getOrCreate()

22/11/24 10:19:45 WARN Utils: Your hostname, codeStation resolves to a loopback address: 127.0.1.1; using 192.168.57.83 instead (on interface wlo1)
22/11/24 10:19:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/24 10:19:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=NYSE>

In [4]:
nyse_path = "/run/media/solverbot/repoA/gitFolders/data-engineering-spark/data/nyse"
de_path = "/run/media/solverbot/repoA/gitFolders/moreDE"
companyList = spark.read.csv(nyse_path+"/companylist_noheader.csv",inferSchema=True)

In [14]:
import shutil
shutil.unpack_archive(nyse_path+"/nyse_data.tar.gz",extract_dir='nyse_year')
nyse_yearly = spark.read.csv(de_path+"/nyse_year",inferSchema=True). \
        toDF("company_name","date","open","high","low","close","volume")
nyse_yearly.show(2)

+------------+-----------+-----+-----+-----+-----+------+
|company_name|       date| open| high|  low|close|volume|
+------------+-----------+-----+-----+-----+-----+------+
|           A|01-Jan-2013|40.94|40.94|40.94|40.94|     0|
|          AA|01-Jan-2013| 8.68| 8.68| 8.68| 8.68|     0|
+------------+-----------+-----+-----+-----+-----+------+
only showing top 2 rows



In [5]:
spark.sql("CREATE DATABASE IF NOT EXISTS nyse_database")
spark.sql("USE nyse_database")

DataFrame[]

In [6]:
#reading from the spark_database
nyse_yearly = spark.read.parquet("spark-warehouse/nyse_database.db/nyse_table_raw")

In [ ]:
spark.sql("""
  SELECT TO_DATE(CAST(UNIX_TIMESTAMP('12/31/2049', 'MM/dd/yyyy') AS TIMESTAMP)) AS newdate"""
).show()

In [18]:
nyse_yearly.select("company_name","date","open","high","low","close",
                "volume",substring(col("date"),8,10).alias('year')).createOrReplaceTempView("nyse_table_raw")

In [19]:
spark.sql("""SELECT * FROM nyse_table_raw LIMIT 5""").show()

+------------+-----------+-----+-----+-----+-----+------+----+
|company_name|       date| open| high|  low|close|volume|year|
+------------+-----------+-----+-----+-----+-----+------+----+
|           A|01-Jan-2013|40.94|40.94|40.94|40.94|     0|2013|
|          AA|01-Jan-2013| 8.68| 8.68| 8.68| 8.68|     0|2013|
|         AAN|01-Jan-2013|28.28|28.28|28.28|28.28|     0|2013|
|         AAP|01-Jan-2013|72.35|72.35|72.35|72.35|     0|2013|
|         AAT|01-Jan-2013|27.93|27.93|27.93|27.93|     0|2013|
+------------+-----------+-----+-----+-----+-----+------+----+



In [22]:
nyse_yearly.select(current_date()). \
    show(2)

+--------------+
|current_date()|
+--------------+
|    2022-11-24|
|    2022-11-24|
+--------------+
only showing top 2 rows



In [23]:
spark.sql("SHOW TABLES").show()

+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|         |nyse_table_raw|       true|
+---------+--------------+-----------+



In [24]:
help(lit)

Help on function lit in module pyspark.sql.functions:

lit(col: Any) -> pyspark.sql.column.Column
    Creates a :class:`~pyspark.sql.Column` of literal value.
    
    .. versionadded:: 1.3.0
    
    Examples
    --------
    >>> df.select(lit(5).alias('height')).withColumn('spark_user', lit(True)).take(1)
    [Row(height=5, spark_user=True)]



In [33]:
nyse_yearly.show(1)

+------------+-----------+-----+-----+-----+-----+------+
|company_name|       date| open| high|  low|close|volume|
+------------+-----------+-----+-----+-----+-----+------+
|           A|01-Jan-2013|40.94|40.94|40.94|40.94|     0|
+------------+-----------+-----+-----+-----+-----+------+
only showing top 1 row



In [43]:
spark.sql("select date_format(date '1970-01-31', 'd')").show()

+---------------------------------+
|date_format(DATE '1970-01-31', d)|
+---------------------------------+
|                               31|
+---------------------------------+



In [46]:
spark.sql("select date_format(date '1970-01-31', 'MMM')").show()

+-----------------------------------+
|date_format(DATE '1970-01-31', MMM)|
+-----------------------------------+
|                                Jan|
+-----------------------------------+



In [45]:
spark.sql("select date_format('1970-Jan-31', 'd')").show()

+---------------------------+
|date_format(1970-Jan-31, d)|
+---------------------------+
|                       null|
+---------------------------+



In [11]:
spark.sql("SELECT year, company_name FROM nyse_table_raw LIMIT 5").show()

+----+------------+
|year|company_name|
+----+------------+
|2013|           A|
|2013|          AA|
|2013|         AAN|
|2013|         AAP|
|2013|         AAT|
+----+------------+



In [27]:
#getting the count of the nyse data for each year
spark.sql("""SELECT COUNT(company_name) as yearlyCount, year
            FROM nyse_table_raw
            GROUP BY year
            ORDER BY COUNT(company_name)""").show()

+-----------+----+
|yearlyCount|year|
+-----------+----+
|     145126|2014|
|     567649|2009|
|     600688|2010|
|     648323|2011|
|     711845|2012|
|     798061|2013|
+-----------+----+



In [29]:
#number of companies listed each year in NYSE
spark.sql("""SELECT COUNT(DISTINCT company_name) as listedCompanies, year
            FROM nyse_table_raw
            GROUP BY year
            ORDER BY COUNT(company_name)""").show()

+---------------+----+
|listedCompanies|year|
+---------------+----+
|           3249|2014|
|           2240|2009|
|           2407|2010|
|           2581|2011|
|           2905|2012|
|           3215|2013|
+---------------+----+



In [30]:
#Count the number of companies listed in 2009
spark.sql("""SELECT DISTINCT company_name, substring(date, 8,10) AS year
                FROM nyse_table_raw
                WHERE substring(date, 8, 10) = '2009'
                ORDER BY company_name DESC""").count()

2240

### Found that table can be loaded but hive support is required
spark.sql("""LOAD DATA LOCAL INPATH '/spark-warehouse/nyse-database.db/nyse_table_raw'
    OVERWRITE INTO TABLE nyse_table_raw""")

In [42]:
help(date_format)

Help on function date_format in module pyspark.sql.functions:

date_format(date: 'ColumnOrName', format: str) -> pyspark.sql.column.Column
    Converts a date/timestamp/string to a value of string in the format specified by the date
    format given by the second argument.
    
    A pattern could be for instance `dd.MM.yyyy` and could return a string like '18.03.1993'. All
    pattern letters of `datetime pattern`_. can be used.
    
    .. _datetime pattern: https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
    
    .. versionadded:: 1.5.0
    
    Notes
    -----
    Whenever possible, use specialized functions like `year`.
    
    Examples
    --------
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(date_format('dt', 'MM/dd/yyy').alias('date')).collect()
    [Row(date='04/08/2015')]



In [54]:
nyse_yearly.select("date").show()

+-----------+
|       date|
+-----------+
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
|01-Jan-2013|
+-----------+
only showing top 20 rows



In [61]:
spark.sql("""SELECT EXTRACT(month FROM date(date))
                FROM nyse_table_raw""").show(5)

+------------------------+
|extract(month FROM date)|
+------------------------+
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
+------------------------+
only showing top 5 rows

